En este endpoint se nos pide la cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [16]:
df_steam_games = pq.read_table('Datos/df_steam_games_limpio_final.parquet').to_pandas()

In [37]:
df_steam_games.head(5)

,genres,app_name,tags,specs,price,id,developer,anio
0,action,lost summoner kitty,strategy,single player,4.99,761140,kotoshiro,2018
1,action,lost summoner kitty,action,single player,4.99,761140,kotoshiro,2018
2,action,lost summoner kitty,indie,single player,4.99,761140,kotoshiro,2018
3,action,lost summoner kitty,casual,single player,4.99,761140,kotoshiro,2018
4,action,lost summoner kitty,simulation,single player,4.99,761140,kotoshiro,2018


Voy a dropear los nulos de las columnas 'id' y 'developer' para que no entorpezcan el desarrollo de la actividad

In [17]:
df_steam_games.dropna(subset=['id'], inplace=True)
df_steam_games.dropna(subset=['developer'], inplace=True)

df_steam_games.tail(5)

,genres,app_name,tags,specs,price,id,developer,anio
2020045,none,maze run vr,vr,htc vive,4.99,681550,none,0
2020046,none,maze run vr,vr,oculus rift,4.99,681550,none,0
2020047,none,maze run vr,vr,tracked motion controllers,4.99,681550,none,0
2020048,none,maze run vr,vr,standing,4.99,681550,none,0
2020049,none,maze run vr,vr,room scale,4.99,681550,none,0


Todavía tengo nulos en la columna 'developer'. 

In [40]:
df_steam_games[df_steam_games['developer'] == 'none'].shape, df_steam_games.shape

((86363, 8), (2020050, 8))

Elimino las filas creadas por el explode

In [39]:
df = df_steam_games.groupby('id').first().reset_index()

In [21]:
df.head()

,id,genres,app_name,tags,specs,price,developer,anio
0,0,none,none,none,none,19.99,none,0
1,10,action,counter strike,action,multi player,9.99,valve,2000
2,20,action,team fortress classic,action,multi player,4.99,valve,1999
3,30,action,day of defeat,fps,multi player,4.99,valve,2003
4,40,action,deathmatch classic,action,multi player,4.99,valve,2001


In [41]:
df['price'] = df['price'].astype(float)

Elimino las columnas que no necesita consultar la query

In [47]:
df.drop(columns=['genres','app_name','tags','specs'], axis=1, inplace=True)

In [48]:
import pyarrow.parquet as pq
df.to_parquet('df_endpoint1.parquet')

In [49]:
df.shape

(30756, 4)

Creo dos dataframes. Cantidad muestra la cantidad de juegos desarrollados por el developer por cada año, y cantidad_free muestra la cantidad de juegos gratis desarrollados por éste por cada año.

In [50]:
cantidad = df[df['developer'] == 'developer'].groupby('anio').count()
cantidad_free = df[(df['developer'] == 'developer') & (df['price'] == 0.0)].groupby('anio').count()

Voy a probar con un ejemplo de developer 'valve'

In [51]:
cantidad = df[df['developer'] == 'valve'].groupby('anio').count()
cantidad_free = df[(df['developer'] == 'valve') & (df['price'] == 0.0)].groupby('anio').count()

In [52]:
df['developer'][df['developer'] == 'valve'].value_counts()

developer
valve    25
Name: count, dtype: int64

In [53]:
cantidad_free

,id,price,developer
anio,,,
2007,1,1,1
2010,1,1,1
2016,1,1,1


In [54]:
cantidad

,id,price,developer
anio,,,
1998,1,1,1
1999,1,1,1
2000,2,2,2
2001,1,1,1
2003,1,1,1
2004,5,5,5
2006,2,2,2
2007,3,3,3
2008,1,1,1


Voy a pasar los datos de los dataframes a listas para poder hacer el diccionario al final.

In [55]:
cantidad_juegos = cantidad['developer'].tolist()
cantidad_juegos_free = cantidad_free['developer'].tolist()

In [56]:
anio = cantidad.index.tolist()
anio_free = cantidad_free.index.tolist()

In [57]:
cantidad_juegos,anio

([1, 1, 2, 1, 1, 5, 2, 3, 1, 1, 2, 1, 1, 1, 2],
 [1998,
  1999,
  2000,
  2001,
  2003,
  2004,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2016,
  2017])

In [58]:
cantidad_juegos_free,anio_free

([1, 1, 1], [2007, 2010, 2016])

Este código usa un for loop para recorrer las lsitas y crear un diccionario con el año como clave, la cantidad de juegos desarrollados por el developer como valor y el porcentaje de juegos que son free.

In [59]:
porc_cont_free = {}
for i in range(len(anio)):
  if anio[i] not in anio_free:
    porc_cont_free[anio[i]] = [cantidad_juegos[i],"0%"]
  else:
    for j in range(len(anio_free)):
      if anio_free[j]==anio[i]:
        porc_cont_free[anio[i]] = [cantidad_juegos[i], cantidad_juegos_free[j]/cantidad_juegos[i]*100]

In [60]:
porc_cont_free

{1998: [1, '0%'],
 1999: [1, '0%'],
 2000: [2, '0%'],
 2001: [1, '0%'],
 2003: [1, '0%'],
 2004: [5, '0%'],
 2006: [2, '0%'],
 2007: [3, 33.33333333333333],
 2008: [1, '0%'],
 2009: [1, '0%'],
 2010: [2, 50.0],
 2011: [1, '0%'],
 2012: [1, '0%'],
 2016: [1, 100.0],
 2017: [2, '0%']}